In [ ]:
import pandas as pd
import os
import pathlib
import re
import ast


In [2]:
path_outsiders = "/export/data_ml4ds/NextProcurement/Junio_2025/pliegosPlace/red_data_outsiders_2024_conTitleCPVLink_chunks"
path_insiders = "/export/data_ml4ds/NextProcurement/Junio_2025/pliegosPlace/red_data_insiders_2024_conTitleCPVLink_chunks"

In [3]:
df_insiders = pd.read_parquet(path_insiders)
print("Insiders data loaded")
df_outsiders = pd.read_parquet(path_outsiders)
print("Outsiders data loaded")

# make id unique
df_insiders["id"] = "I" + df_insiders["id"].astype(str)
df_outsiders["id"] = "O" + df_outsiders["id"].astype(str)

df_in_out = pd.concat([df_insiders, df_outsiders], ignore_index=True)

Insiders data loaded
Outsiders data loaded
Outsiders data loaded


In [10]:
df_in_out = df_in_out.rename(columns={"ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv"})

In [11]:
df_in_out.columns

Index(['place_id', 'link', 'cpv', 'title', 'url', 'id', 'resultado_tecnico',
       'path_tecnico', 'resultado_administrativo', 'path_administrativo',
       'texto_tecnico', 'texto_administrativo'],
      dtype='object')

In [5]:
print(df_in_out["id"].duplicated().sum())
assert len(df_in_out) == len(df_in_out["id"].unique()) == len(df_insiders) + len(df_outsiders)
print(len(df_outsiders))
print(len(df_insiders))

0
35340
114600


# Failure cases

In [8]:
import json

path_js = "/export/usuarios_ml4ds/lbartolome/Repos/patchwork/generative_results.json"

with open(path_js, "r") as f:
    js = f.read()
data = json.loads(js)
df_js = pd.json_normalize(data)

erros = df_js[df_js.row_index.isin([4,46, 128,164,165,208,234,288,317,326,375,409,456])].gold.values.tolist()


In [16]:
erros

['Contrato de servicios denominado desinfección, desinsectación, desratización, prevención y control de otras plagas y prevención de legionelosis en el municipio de Sagunto.',
 'T.A. 2025 - PAN3 - Suministro de medio de transporte de personal de gran capacidad para el Parque de Autos Nº3',
 'Suministro de material promocional para la II Jornada de Formación “Ética e integridad en la Contratación Pública” en Calp (Alicante) - Lote 1 – Acreditaciones.',
 'Servicio de formación en el uso de equipos de cardio protección (desfibriladores) en los centros educativos dependientes del IMEB',
 'Suministro de combustible, por lotes, para las embarcaciones del grupo de rescate acuáticos del SEIS (lote 1) y para la embarcación del Aquarium Finisterrae (lote 2)',
 'Servicio De Control De Fauna En El Aeropuerto Internacional Region De Murcia',
 'Suministro de pupitres unipersonales M-19, con silla para educación secundaria y bachillerato, mobiliario homologado con destino a centros docentes de titula

In [14]:
# keep rows from df_in_out whose title is in erros
df_final = df_in_out[df_in_out["title"].isin(erros)].reset_index(drop=True)
df_final.to_parquet("/export/data_ml4ds/NextProcurement/pruebas_oct_2025/objective_extractor/data/insiders_outsiders_2024_with_errors.parquet")
len(df_final)

44

In [15]:
df_final.texto_tecnico

0      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
1      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
2      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
3      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
4      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
5      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
6      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
7      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
8      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
9      \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
10     \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
11     \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
12     \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
13     \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
14     \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
15     \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
16     \n \nGrupo Tragsa (Grupo SEPI) - Sede Social:...
17     \n \nGrupo Tragsa (Grupo SEPI) - Sede Soc

# Objective extractor experiments

We get a random sample of 500 outsiders + 500 insiders

In [10]:
ERR = "[ERROR: PDF sin texto extraíble (posiblemente escaneado)]"

text_col = df_in_out["texto_tecnico"].astype("string").fillna("")
id_str   = df_in_out["id"].astype("string").fillna("")

mask_I = id_str.str.startswith("I")
mask_O = id_str.str.startswith("O")
mask_ok_text = ~text_col.str.strip().str.startswith('[ERROR:')

# rows that are entirely control chars / whitespace (at least one char)
mask_all_control = ~text_col.str.fullmatch(r'[\x00-\x1F\x7F\s]+', na=False)

# rows whose title column is not nan or empty after stripping
title_col = df_in_out["title"].astype("string").fillna("")
mask_title_ok = ~title_col.str.strip().eq("")

df_I = df_in_out.loc[mask_I & mask_all_control & mask_ok_text & mask_title_ok]
df_O = df_in_out.loc[mask_O & mask_all_control & mask_ok_text & mask_title_ok]

print(f"Available insiders after filtering: {len(df_I)}")
print(f"Available outsiders after filtering: {len(df_O)}")

# Strategy 1: Ensure exactly 1000 total samples
target_total = 1000
target_I = min(int(target_total/2), len(df_I))
target_O = min(int(target_total/2), len(df_O))

# If we don't have enough from one category, compensate with the other
if target_I + target_O < target_total:
    # Not enough samples total, take what we can get
    print(f"Warning: Only {target_I + target_O} samples available, less than target {target_total}")
elif target_I < target_total/2:
    # Not enough insiders, take more outsiders
    remaining = target_total - target_I
    target_O = min(remaining, len(df_O))
    print(f"Adjusted: Taking {target_I} insiders and {target_O} outsiders")
elif target_O < target_total/2:
    # Not enough outsiders, take more insiders  
    remaining = target_total - target_O
    target_I = min(remaining, len(df_I))
    print(f"Adjusted: Taking {target_I} insiders and {target_O} outsiders")

# Sample the data
sample_I = df_I.sample(target_I, random_state=42) if target_I > 0 else df_I.head(0)
sample_O = df_O.sample(target_O, random_state=42) if target_O > 0 else df_O.head(0)

df_sample = pd.concat([sample_I, sample_O], ignore_index=True)

print(f"Final sample size: {len(df_sample)} (Insiders: {len(sample_I)}, Outsiders: {len(sample_O)})")

path_save = "/export/data_ml4ds/NextProcurement/pruebas_oct_2025/objective_extractor/data/insiders_outsiders_{}_{}.parquet"

os.makedirs(pathlib.Path(path_save).parent, exist_ok=True)
df_sample.to_parquet(path_save.format(target_I, target_O), index=False)

Available insiders after filtering: 109917
Available outsiders after filtering: 33474
Final sample size: 1000 (Insiders: 500, Outsiders: 500)


In [10]:
df_sample[df_sample['texto_tecnico'].str.startswith('[ERROR:')].texto_tecnico

233    [ERROR: No se pudo procesar el PDF - FileDataE...
322    [ERROR: No se pudo procesar el PDF - FileDataE...
395    [ERROR: No se pudo procesar el PDF - FileDataE...
419    [ERROR: No se pudo procesar el PDF - FileDataE...
472    [ERROR: No se pudo procesar el PDF - FileDataE...
540    [ERROR: No se pudo procesar el PDF - FileDataE...
568    [ERROR: No se pudo procesar el PDF - FileDataE...
699    [ERROR: No se pudo procesar el PDF - FileDataE...
896    [ERROR: No se pudo procesar el PDF - FileDataE...
929    [ERROR: No se pudo procesar el PDF - FileDataE...
Name: texto_tecnico, dtype: object

In [7]:
df_sample


,place_id,link,ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode,title,url,id,resultado_tecnico,path_tecnico,resultado_administrativo,path_administrativo,texto_tecnico,texto_administrativo
0,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,[71356200.0],Elaboración y redacción del Plan de Movilidad ...,{'administrativo': 'https://contrataciondelest...,I86740,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Firmado por:\nCYNTHIA RODRIGUEZ HERNANDEZ - Ar...,Firmado por:\nCANDELARIA YURENA GONZÁLEZ BARRI...
1,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,[39298900.0],Contratación del suministro de flores y planta...,{'administrativo': 'https://contrataciondelest...,I111100,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n1 \nCopyright ©2024 Paradores de Turismo...,\n \n1 \nCopyright ©2024 Paradores de Turismo...
2,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,[85148000.0],Contratación del servicio de Análisis Clínicos...,{'administrativo': 'https://contrataciondelest...,I69774,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \n \n029-2024-0215 Avda...,\n \n \n \n \n029-2024-0215 Avda. Ti...
3,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,[92000000.0],Objeto del contrato: La finalidad del presente...,{'administrativo': 'https://contrataciondelest...,I69639,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Ayuntamiento de Aldeamayor de San Martín\nRESU...,\nAyuntamiento de Aldeamayor de San Martín\nR...
4,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,[35113200.0],CEMILVET - Suministro de instalación de 2 auto...,{'administrativo': 'https://contrataciondelest...,I30954,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \nPágina 1 de 3 \n \n \nPLIEGO DE PRESCRIP...,MINISTERIO DE DEFENSA\nSubsecretaría de Defens...
...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://contrataciondelestado.es/sindicacion/P...,https://contractaciopublica.cat/ca/detall-publ...,"[[71317210, 71000000]]",MRU Serveis de redacció de l'estudi de seguret...,{'administrativo': 'https://contractaciopublic...,O5869,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \nEXA: 47/2024 \n \n \n \n \n \n \n ...,\nServeis \nProcediment Obert SARHA \nDEF-CC...
996,https://contrataciondelestado.es/sindicacion/P...,https://contractaciopublica.cat/ca/detall-publ...,[55523100.0],Servei de menjador de l'Escola de Rellinars,{'administrativo': 'https://contractaciopublic...,O12850,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Núm. Expedient:08067181/2024/1\nPLEC DE PRESCR...,Logotip del centre \nExp. 08067181/2024/01 \n ...
997,https://contrataciondelestado.es/sindicacion/P...,https://www.contratacion.euskadi.eus/webkpe00-...,[48730000.0],Suministros y servicios para la protección y c...,{'administrativo': 'https://www.contratacion.e...,O24066,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n1 \n \n \n \n \n \n \n \n \n \nEJIE-147-202...,\n \n \n

# Divide by CPV and get CPV5 and CPV8

In [22]:
import numpy as np
import ast
import pandas as pd

def safe_parse_possible_array_string(item):
    if isinstance(item, np.ndarray) and len(item) == 1:
        string = item[0]
        if isinstance(string, str) and string.strip().lower() == "nan":
            return []
        try:
            parsed = ast.literal_eval(string)
            if isinstance(parsed, list):
                return parsed
        except (ValueError, SyntaxError):
            return None
    return None

def extract_first_two_digits(code):
    try:
        code_str = str(int(float(code)))
        return code_str[:2] if len(code_str) >= 2 else None
    except (ValueError, TypeError):
        return None

def extract_code_length(code):
    try:
        code_str = str(int(float(code)))
        return len(code_str)
    except (ValueError, TypeError):
        return None

def extract_cpv_depth_length(code):
    """Length after stripping trailing zeros (the 'depth' you summarized)."""
    try:
        code_str = str(int(float(code)))
        return len(code_str.rstrip('0'))
    except (ValueError, TypeError):
        return None

def analyze_cpv_depths(df, column='cpv'):
    depths_flat = []
    format_issues = []
    nan_count = 0
    total = 0

    first_two_digits_all = []
    lengths_all = []
    depths_all = []  # NEW: per-row depth list

    for item in df[column]:
        first_two_digits_row, lengths_row, depths_row = [], [], []

        if isinstance(item, np.ndarray) and len(item) == 1 and str(item[0]).strip().lower() == "nan":
            nan_count += 1
            first_two_digits_all.append(None)
            lengths_all.append(None)
            depths_all.append(None)
            continue

        iterable = None
        if isinstance(item, list):
            iterable = item if item else None
        elif pd.isna(item):
            nan_count += 1
        else:
            recovered_list = safe_parse_possible_array_string(item)
            iterable = recovered_list if recovered_list is not None else [item]

        if iterable is None:
            first_two_digits_all.append(None)
            lengths_all.append(None)
            depths_all.append(None)
            continue

        for code in iterable:
            total += 1

            depth_len = extract_cpv_depth_length(code)
            if depth_len is not None:
                depths_flat.append(depth_len)
                depths_row.append(depth_len)
            else:
                format_issues.append(code)

            two = extract_first_two_digits(code)
            if two is not None:
                first_two_digits_row.append(two)

            raw_len = extract_code_length(code)
            if raw_len is not None:
                lengths_row.append(raw_len)

        first_two_digits_all.append(first_two_digits_row or None)
        lengths_all.append(lengths_row or None)
        depths_all.append(depths_row or None)

    # Attach columns
    df[column + '_first_two_digits'] = first_two_digits_all
    df[column + '_length'] = lengths_all           # raw digit count (usually 8)
    df[column + '_depth'] = depths_all             # NEW: trailing-zero–stripped length

    # Summary like before
    depth_counts = pd.Series(depths_flat).value_counts().sort_index()
    print(f"Total CPV codes processed: {total}")
    print(f"Format issues: {len(format_issues)}")
    print(f"NaNs or empty lists: {nan_count}")
    print(f"Valid CPV codes with depth: {len(depths_flat)}")
    return depth_counts, format_issues, nan_count, total

depth_counts, bad_cpvs, nan_count, total_cpvs = analyze_cpv_depths(df_in_out)


print(depth_counts)
print("\nExamples of format issues:")
print(bad_cpvs[:10])
print("\nPreview of new columns:")
print(df_in_out[['cpv', 'cpv_first_two_digits', 'cpv_length']].head())


/tmp/ipykernel_1390046/2401434481.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  code_str = str(int(float(code)))
/tmp/ipykernel_1390046/2401434481.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  code_str = str(int(float(code)))
/tmp/ipykernel_1390046/2401434481.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  code_str = str(int(float(code)))


Total CPV codes processed: 243808
Format issues: 0
NaNs or empty lists: 1255
Valid CPV codes with depth: 243808
1     4717
2    17393
3    37113
4    48946
5    56121
6    47802
7    21053
8    10663
Name: count, dtype: int64

Examples of format issues:
[]

Preview of new columns:
                      cpv cpv_first_two_digits cpv_length
0            [66511000.0]                 [66]        [8]
1            [48821000.0]                 [48]        [8]
2            [79995000.0]                 [79]        [8]
3  [[50118110, 60100000]]             [50, 60]     [8, 8]
4  [[50300000, 50330000]]             [50, 50]     [8, 8]


In [23]:
df_depth_8 = df_in_out[
    df_in_out['cpv_depth'].apply(lambda x: (isinstance(x, list) and 8 in x) or (x == 8))
]
df_depth_5 = df_in_out[
    df_in_out['cpv_depth'].apply(lambda x: (isinstance(x, list) and 5 in x) or (x == 5))
]

print("Rows with depth 8:", len(df_depth_8))
print("Rows with depth 5:", len(df_depth_5))

Rows with depth 8: 9025
Rows with depth 5: 45084


In [24]:
import dask.dataframe as da

ddf = da.from_pandas(df_depth_8, chunksize=400)
path_save_8 = "/export/data_ml4ds/NextProcurement/PLACE/to_process_cpv8"
ddf.to_parquet(path_save_8)

ddf = da.from_pandas(df_depth_5, chunksize=400)
path_save_5 = "/export/data_ml4ds/NextProcurement/PLACE/to_process_cpv5"
ddf.to_parquet(path_save_5)